## Tensorflow怎样保存与加载模型

#### 背景：模型训练和预估服务
* 训练模型一般不是目标，在线预估服务才是
* 一般会有一个程序训练模型，训练好之后把模型保存，在线预估加载模型实现服务
* 模型一般包括模型结构、训练好的模型参数两部分组成

#### 保存模型的方法
* 模型结构+参数打包保存和加载
* 模型结构存储到json文件，模型参数保存到h5文件
* 模型结构存储到yaml文件，模型参数保存到h5文件

其实，模型参数甚至也会存储到redis、mysql等，各个web服务可以随意更新加载

In [1]:
import pandas as pd
import tensorflow as tf

### 1. 准备和训练一个模型

#### 读取数据

In [2]:
df = pd.read_csv("./datas/heart/heart.csv")

# 把thal列变成数字编码
df['thal'] = pd.Categorical(df['thal'])
df['thal'] = df['thal'].cat.codes

# 要预测的目标，这是个二分类问题
target = df.pop('target')

#### 构建dataset

In [3]:
# 构建dataset，其实是把pandas数据转换成numpy数组进行转换的
dataset = tf.data.Dataset.from_tensor_slices((df.values, target.values))
# Shuffle and batch the dataset.
train_dataset = dataset.shuffle(len(df)).batch(4)

In [4]:
# 用于一会的测试
for x, y in train_dataset.take(1):
    input_data = x.numpy()
    print(input_data)

[[ 62.    0.    3.  130.  263.    0.    0.   97.    0.    1.2   2.    1.
    4. ]
 [ 43.    1.    4.  150.  247.    0.    0.  171.    0.    1.5   1.    0.
    3. ]
 [ 76.    0.    3.  140.  197.    0.    1.  116.    0.    1.1   2.    0.
    3. ]
 [ 63.    1.    1.  145.  233.    1.    2.  150.    0.    2.3   3.    0.
    2. ]]


#### 搭建训练模型

In [5]:
model = tf.keras.Sequential([
    tf.keras.layers.Dense(10, input_shape=(df.shape[1],)),
    tf.keras.layers.Dense(10, activation='relu'),
    tf.keras.layers.Dense(1)
])

model.compile(optimizer='adam',
            loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
            metrics=['accuracy'])

model.fit(train_dataset, epochs=10)

Epoch 1/10
76/76 [==============================] - 1s 15ms/step - loss: 6.6763 - accuracy: 0.5380
Epoch 2/10
76/76 [==============================] - 0s 1ms/step - loss: 2.0135 - accuracy: 0.6271
Epoch 3/10
76/76 [==============================] - 0s 1ms/step - loss: 0.8384 - accuracy: 0.7096
Epoch 4/10
76/76 [==============================] - 0s 2ms/step - loss: 0.4721 - accuracy: 0.7492
Epoch 5/10
76/76 [==============================] - 0s 2ms/step - loss: 0.5079 - accuracy: 0.7459
Epoch 6/10
76/76 [==============================] - 0s 2ms/step - loss: 0.4660 - accuracy: 0.7624
Epoch 7/10
76/76 [==============================] - 0s 2ms/step - loss: 0.4762 - accuracy: 0.7690
Epoch 8/10
76/76 [==============================] - 0s 2ms/step - loss: 0.4430 - accuracy: 0.7756
Epoch 9/10
76/76 [==============================] - 0s 2ms/step - loss: 0.5149 - accuracy: 0.7954
Epoch 10/10
76/76 [==============================] - 0s 2ms/step - loss: 0.4903 - accuracy: 0.7921


### 方法1：把模型结构和模型参数一起保存

In [6]:
model.save("./models/heart_model_method1.h5")

In [7]:
model_total = tf.keras.models.load_model("./models/heart_model_method1.h5")

In [8]:
# 一个batch
input_data.shape

(4, 13)

In [9]:
model_total.predict(input_data)

array([[-0.32006454],
       [-4.5661125 ],
       [-1.7438908 ],
       [-2.3644123 ]], dtype=float32)

### 方法2：模型结构保存到json，模型参数保存到h5

In [10]:
# 将模型结构保存到json文件
open("./models/heart_model_json.json", "w").write(model.to_json())

1434

In [11]:
# 将模型参数保存到.h5文件
model.save_weights("./models/heart_model_json.h5")

In [12]:
# 加载模型
model_json = tf.keras.models.model_from_json(open("./models/heart_model_json.json").read())
model_json.load_weights("./models/heart_model_json.h5")

In [13]:
# 注意需要重新编译模型
model_json.compile(optimizer='adam',
            loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
            metrics=['accuracy'])

In [14]:
# 实现预估
model_json.predict(input_data)

array([[-0.32006454],
       [-4.5661125 ],
       [-1.7438908 ],
       [-2.3644123 ]], dtype=float32)

### 方法3：模型结构保存到yaml，模型参数保存到h5

In [15]:
# 将模型结构保存到yaml文件
open("./models/heart_model_yaml.yaml", "w").write(model.to_yaml())

1591

In [16]:
# 将模型参数保存到.h5文件
model.save_weights("./models/heart_model_yaml.h5")

In [17]:
# 加载模型
model_yaml = tf.keras.models.model_from_yaml(open("./models/heart_model_yaml.yaml").read())
model_yaml.load_weights("./models/heart_model_yaml.h5")

/Users/peishuaishuai/anaconda3/lib/python3.7/site-packages/tensorflow_core/python/keras/saving/model_config.py:76: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  config = yaml.load(yaml_string)


In [18]:
# 注意需要重新编译模型
model_yaml.compile(optimizer='adam',
            loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
            metrics=['accuracy'])

In [19]:
# 实现预估
model_yaml.predict(input_data)

array([[-0.32006454],
       [-4.5661125 ],
       [-1.7438908 ],
       [-2.3644123 ]], dtype=float32)

In [21]:
input_data

array([[ 62. ,   0. ,   3. , 130. , 263. ,   0. ,   0. ,  97. ,   0. ,
          1.2,   2. ,   1. ,   4. ],
       [ 43. ,   1. ,   4. , 150. , 247. ,   0. ,   0. , 171. ,   0. ,
          1.5,   1. ,   0. ,   3. ],
       [ 76. ,   0. ,   3. , 140. , 197. ,   0. ,   1. , 116. ,   0. ,
          1.1,   2. ,   0. ,   3. ],
       [ 63. ,   1. ,   1. , 145. , 233. ,   1. ,   2. , 150. ,   0. ,
          2.3,   3. ,   0. ,   2. ]])

In [25]:
import pandas as pd
pd.DataFrame(input_data).to_json(orient='values')

'[[62.0,0.0,3.0,130.0,263.0,0.0,0.0,97.0,0.0,1.2,2.0,1.0,4.0],[43.0,1.0,4.0,150.0,247.0,0.0,0.0,171.0,0.0,1.5,1.0,0.0,3.0],[76.0,0.0,3.0,140.0,197.0,0.0,1.0,116.0,0.0,1.1,2.0,0.0,3.0],[63.0,1.0,1.0,145.0,233.0,1.0,2.0,150.0,0.0,2.3,3.0,0.0,2.0]]'